In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/data.xlsx")

In [ ]:
data.shape

(991780, 69)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991780 entries, 0 to 991779
Data columns (total 69 columns):
 #   Column                                           Non-Null Count   Dtype         
---  ------                                           --------------   -----         
 0   Product_x                                        991780 non-null  object        
 1   UPC 13 digit                                     991780 non-null  int64         
 2   Unit Sales                                       991780 non-null  int64         
 3   Unit Sales No Merch                              991780 non-null  int64         
 4   Unit Sales Any Merch                             991780 non-null  int64         
 5   Unit Sales Price Reductions Only                 991780 non-null  int64         
 6   Unit Sales Feature Only                          991780 non-null  int64         
 7   Unit Sales Display Only                          991780 non-null  int64         
 8   Unit Sales Feature and D

In [ ]:
ppu = data.groupby('Brand Name')[['Price per Unit', 'Unit Sales', 'Volume Sales', 'Dollar Sales', 'Price per Volume']].sum().reset_index()

In [ ]:
ppu = ppu[ppu['Brand Name'] != "PRIVATE LABEL"]

In [ ]:
ppu.sort_values(by="Dollar Sales", ascending=False).head(10)

,Brand Name,Price per Unit,Unit Sales,Volume Sales,Dollar Sales,Price per Volume
120,MORNINGSTAR FARMS,364412.138517,426113124,2.809315e+08,2.059644e+09,463476.629765
91,IMPOSSIBLE,159024.742665,141971772,1.099315e+08,1.015503e+09,182237.025310
66,GARDEIN,383076.399487,195622201,1.455550e+08,9.404258e+08,484824.813519
15,BEYOND MEAT BEYOND BURGER,37414.689097,85121181,5.079289e+07,4.844174e+08,39140.157664
13,BEYOND MEAT,87534.903420,63255071,5.787699e+07,4.811132e+08,102625.711542
16,BEYOND MEAT BEYOND SAUSAGE,44061.232856,46290870,4.050692e+07,3.660306e+08,50355.694693
190,TOFURKY,216693.352103,84177343,4.221858e+07,3.533746e+08,355157.539947
54,FIELD ROAST,179603.214026,50352061,3.851213e+07,2.938434e+08,268884.709455
23,BOCA,159285.834235,72270200,4.921962e+07,2.850745e+08,181443.607858
123,MORNINGSTAR FARMS GRILLERS,30183.444962,50783248,3.783811e+07,2.492869e+08,37405.350358


In [ ]:
import plotly.express as px
import plotly.io as pio

# Set a specific theme (e.g., ggplot2)
pio.templates.default = "seaborn"

# Create the bar chart
fig = px.bar(
    ppu.sort_values(by="Price per Unit", ascending=False).head(5),
    x="Brand Name",
    y="Price per Unit",
    title="Top 5 Brands by Price per Unit",
    labels={"Price per Unit": "Price per Unit ($)", "Brand Name": "Brand Name"},
    text="Price per Unit"
)

# Update layout for better visuals
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title="Brand Name",
    yaxis_title="Price per Unit ($)",
    showlegend=False
)

fig.show()


In [ ]:
import plotly.express as px
fig = px.bar(
    ppu.sort_values(by="Unit Sales", ascending=False).head(5),
    x="Brand Name",
    y="Unit Sales",
    title="Top 5 Brands by Unit Sales",
    labels={"Unit Sales": "No. of Units", "Brand Name": "Brand Name"},
    text="Unit Sales"
)

# Update layout for better visuals
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title="Brand Name",
    yaxis_title="No. of Units",
    showlegend=False
)

fig.show()

In [ ]:
import plotly.express as px
fig = px.bar(
    ppu.sort_values(by="Dollar Sales", ascending=False).head(5),
    x="Brand Name",
    y="Dollar Sales",
    title="Top 5 Brands by Dollar Sales",
    labels={"Dollar Sales": "Sales in ($)", "Brand Name": "Brand Name"},
    text="Dollar Sales"
)

# Update layout for better visuals
fig.update_traces(texttemplate='$%{y:.2f}', textposition='outside')
fig.update_layout(
    xaxis_title="Brand Name",
    yaxis_title="Sales in ($)",
    showlegend=False
)
fig.show()

In [ ]:
# Group by Region and Brand Name
region_brand_df = data.groupby(['Region', 'Brand Name'])[['Unit Sales', 'Volume Sales', 'Dollar Sales']].sum().reset_index()

# Display the resulting DataFrame
print(region_brand_df.head())

        Region      Brand Name  Unit Sales   Volume Sales  Dollar Sales
0  California   ABBOTS BUTCHER       87619   54841.674849  6.175479e+05
1  California   ACTUAL VEGGIES       56073   39704.945933  4.460241e+05
2  California        AJINOMOTO         853     186.304955  4.779928e+03
3  California             AKUA        6898    2899.618433  5.360339e+04
4  California            ALPHA      285458  203175.374447  1.593488e+06


In [ ]:
top_brands_per_region = region_brand_df.loc[region_brand_df.groupby('Region')['Dollar Sales'].idxmax()]

In [ ]:
import plotly.express as px

# Filter the data for Gardein
gardein_region_df = region_brand_df[region_brand_df['Brand Name'] == 'GARDEIN']

# Create a pie chart for Dollar Sales by Region for Gardein
fig_pie = px.pie(
    gardein_region_df,
    names='Region',
    values='Dollar Sales',
    title='Dollar Sales by Region for Gardein',
    labels={'Dollar Sales': 'Dollar Sales ($)', 'Region': 'Region'},
    hole=0.4  # Optional: creates a donut-style chart
)

fig_pie.update_traces(textinfo='percent+label')
fig_pie.show()


In [ ]:
#top selling forms from GARDEIN
form_df = data[data['Brand Name'] == 'GARDEIN'].groupby('Form')[['Unit Sales', 'Volume Sales', 'Dollar Sales']].sum().reset_index()
form_df = form_df.sort_values(by="Dollar Sales", ascending=False).head(10)

In [ ]:
pio.templates.default = "seaborn"
fig = px.bar(
    form_df,
    x='Form',
    y='Dollar Sales',
    title='Dollar Sales by Product Form',
    labels={'Dollar Sales': 'Dollar Sales ($)', 'Form': 'Product Form'},
    text='Dollar Sales'
)

# Update layout for better visuals
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, showlegend=False)
fig.show()

In [ ]:
#Group data by date
date_df = data.groupby('Date')[['Unit Sales', 'Volume Sales', 'Dollar Sales']].sum().reset_index()
date_df['Date'] = pd.to_datetime(date_df['Date'])

In [ ]:
import plotly.io as pio

# Set a built-in Plotly theme (e.g., "plotly_dark", "ggplot2", "simple_white", "seaborn")
pio.templates.default = "seaborn"

# Create a time series line chart for Dollar Sales
fig = px.line(
    date_df,
    x='Date',
    y='Dollar Sales',
    title='Dollar Sales over Years',
    labels={'Dollar Sales': 'Dollar Sales ($)', 'Date': 'Date'},
    markers=True
)

# Update layout for better visuals
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Sales Metrics',
    legend_title='Metrics',
    title_font=dict(size=20, family='Arial'),
    font=dict(size=14, family='Arial')
)
fig.show()

In [ ]:
#Group data by date and brand name
date_brand_df = data.groupby(['Date', 'Brand Name'])[['Unit Sales', 'Volume Sales', 'Dollar Sales']].sum().reset_index()
date_brand_df['Date'] = pd.to_datetime(date_brand_df['Date'])

In [ ]:
import plotly.express as px

# Filter the data for 'GARDEIN' and 'MORNINGSTAR FARMS'
filtered_time_series = date_brand_df[
    date_brand_df['Brand Name'].isin(['GARDEIN', 'MORNINGSTAR FARMS'])
]

# Create a time series line chart for Dollar Sales for both brands
fig = px.line(
    filtered_time_series,
    x='Date',
    y='Dollar Sales',
    color='Brand Name',
    title='Dollar Sales over Time for GARDEIN and MORNINGSTAR FARMS',
    labels={'Dollar Sales': 'Dollar Sales ($)', 'Date': 'Date', 'Brand Name': 'Brand'},
    markers=True
)

# Update layout for better visuals
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Dollar Sales ($)',
    legend_title='Brand',
    title_font=dict(size=20, family='Arial'),
    font=dict(size=14, family='Arial'),
    template='ggplot2'  # Apply a theme
)

fig.show()

In [ ]:
data['Form'].unique()

array(['GROUND', 'BALL', 'BURGER PATTY', 'CRUMBLE', 'PATTY', 'BURGER',
       'LINK', 'MEATBALL', 'PIECE', 'SLICED', 'STEAK', 'SPARE RIBS',
       'CHUNK', 'TIP', 'SLICE', 'BREAKFAST PATTY', 'FILET', 'SLIDER',
       'CAKE', 'STRIP', 'BITE', 'POPPER', 'BREAKFAST SAUSAGE PATTY',
       'HOT DOG', 'PULLED', 'BREAKFAST LINK', 'FRANK', 'ROAST', 'WHOLE',
       'STICK', 'NOT STATED ON PACKAGE', 'BLOCK', 'ROLL', 'NUGGET',
       'BREAST', 'FINGER', 'TENDER', 'WING', 'CUTLET', 'DICED', 'POPCORN',
       'DIPPER', 'BREAKFAST BITES', 'CHUB', 'DINNER SAUSAGE LINK', 'CUBE',
       'BAR', 'LOAF', 'MEAT LOAF', 'BRATWURST', 'BREAKFAST SAUSAGE ROLL',
       'SPLIT ROPE', 'ROPE', 'ULTRA THIN SLICE', 'DELI SLICED', 'CUT',
       'BREAKFAST SAUSAGE LINK', 'GIZZARD', 'RIBLET', 'SAUSAGE',
       'DINNER LINK', 'DRUMSTICK', 'FUN NUGGETS', 'SHREDS', 'SHREDDED',
       'FRIES', 'TENDERS', 'BINGS', 'SAUSAGE PATTY', 'BAO BUN'],
      dtype=object)

In [ ]:
replacements = {
    # BURGER forms
    'BURGER PATTY': 'BURGER',
    'PATTY': 'BURGER',
    'SLIDER': 'BURGER',

    # BREAKFAST forms
    'BREAKFAST PATTY': 'BREAKFAST SAUSAGE',
    'BREAKFAST SAUSAGE PATTY': 'BREAKFAST SAUSAGE',
    'BREAKFAST BITES': 'BREAKFAST SAUSAGE',
    'BREAKFAST LINK': 'BREAKFAST SAUSAGE LINK',
    'BREAKFAST SAUSAGE ROLL': 'BREAKFAST SAUSAGE',
    'BREAKFAST SAUSAGE LINK': 'BREAKFAST SAUSAGE LINK',

    # NUGGET forms
    'NUGGET': 'FUN NUGGETS',
    'FUN NUGGETS': 'NUGGET',
    'POPCORN': 'NUGGET',
    'FRIES': 'NUGGET',

    # SAUSAGE forms
    'DINNER SAUSAGE LINK': 'SAUSAGE LINK',
    'DINNER LINK': 'SAUSAGE LINK',
    'SAUSAGE PATTY': 'SAUSAGE',
    'BRATWURST': 'SAUSAGE',
    'ROPE': 'SAUSAGE',
    'SPLIT ROPE': 'SAUSAGE',
    'CHUB': 'ROLL',

    # RIB forms
    'RIBLET': 'RIBS',
    'SPARE RIBS': 'RIBS',

    # STEAK forms
    'FILET': 'STEAK',
    'TIP': 'STEAK',

    # CHUNK forms
    'PIECE': 'CHUNK',
    'CHUNK': 'CHUNK',
    'CUBE': 'CHUNK',

    # SLICED forms
    'SLICE': 'SLICED',
    'STRIP': 'SLICED',
    'ULTRA THIN SLICE': 'SLICED',
    'DELI SLICED': 'SLICED',

    # TENDER forms
    'BREAST': 'TENDER',
    'FINGER': 'TENDER',
    'TENDERS': 'TENDER',
    'DIPPER': 'TENDER',

    # OTHER forms
    'HOT DOG': 'FRANK',
    'MEAT LOAF': 'MEAT LOAF',
    'LOAF': 'MEAT LOAF',
    'GROUND': 'GROUND',
    'BALL': 'BALL',
    'BLOCK': 'BLOCK',
    'BAR': 'BLOCK',
    'ROAST': 'ROAST',
    'WHOLE': 'WHOLE',
    'CUT': 'DICED',
    'DICED': 'DICED',
    'CAKE': 'PIECE',
    'BAO BUN': 'BUN',
    'GIZZARD': 'GIZZARD',
    'BITE': 'BITE',
    'POPPER': 'POPPER',
    'LINK': 'LINK',
    'MEATBALL': 'MEATBALL',
    'CRUMBLE': 'CRUMBLE',
    'DRUMSTICK': 'DRUMSTICK',
    'WING': 'WING',
    'ROPE': 'ROPE',
    'SHREDS': 'SHREDDED',
    'SHREDDED': 'SHREDDED',
    'NOT STATED ON PACKAGE': 'UNKNOWN',
    'BINGS': 'UNKNOWN',
}

In [ ]:
data['Form'] = data['Form'].replace(replacements)

In [ ]:
data['Form'].unique()

array(['GROUND', 'BALL', 'BURGER', 'CRUMBLE', 'LINK', 'MEATBALL', 'CHUNK',
       'SLICED', 'STEAK', 'RIBS', 'BREAKFAST SAUSAGE', 'PIECE', 'BITE',
       'POPPER', 'FRANK', 'PULLED', 'BREAKFAST SAUSAGE LINK', 'ROAST',
       'WHOLE', 'STICK', 'UNKNOWN', 'BLOCK', 'ROLL', 'FUN NUGGETS',
       'TENDER', 'WING', 'CUTLET', 'DICED', 'NUGGET', 'SAUSAGE LINK',
       'MEAT LOAF', 'SAUSAGE', 'ROPE', 'GIZZARD', 'DRUMSTICK', 'SHREDDED',
       'BUN'], dtype=object)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991780 entries, 0 to 991779
Data columns (total 69 columns):
 #   Column                                           Non-Null Count   Dtype         
---  ------                                           --------------   -----         
 0   Product_x                                        991780 non-null  object        
 1   UPC 13 digit                                     991780 non-null  int64         
 2   Unit Sales                                       991780 non-null  int64         
 3   Unit Sales No Merch                              991780 non-null  int64         
 4   Unit Sales Any Merch                             991780 non-null  int64         
 5   Unit Sales Price Reductions Only                 991780 non-null  int64         
 6   Unit Sales Feature Only                          991780 non-null  int64         
 7   Unit Sales Display Only                          991780 non-null  int64         
 8   Unit Sales Feature and D

In [ ]:
data['Region'].unique()

array(['Total US ', 'California ', 'Great Lakes ', 'Mid', 'Northeast ',
       'Plains ', 'South Central ', 'Southeast ', 'West '], dtype=object)

In [ ]:
data['Region'].unique()

array(['Total US ', 'California ', 'Great Lakes ', 'Mid', 'Northeast ',
       'Plains ', 'South Central ', 'Southeast ', 'West '], dtype=object)

In [ ]:
data['Total Ounces'] = np.round(data['Total Ounces']).astype(int)
data['Total Ounces'] = data['Total Ounces'].astype('category')

In [ ]:
# Extract the week value from the "Time" column and store it as a categorical column
data['Week'] = data['Date'].dt.isocalendar().week  # Extract week
data['Week'] = data['Week'].astype('category')

# Extract the year value from the "Time" column and store it as a categorical column
data['Year'] = data['Date'].dt.isocalendar().year  # Extract year
data['Year'] = data['Year'].astype('category')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991780 entries, 0 to 991779
Data columns (total 71 columns):
 #   Column                                           Non-Null Count   Dtype         
---  ------                                           --------------   -----         
 0   Product_x                                        991780 non-null  object        
 1   UPC 13 digit                                     991780 non-null  int64         
 2   Unit Sales                                       991780 non-null  int64         
 3   Unit Sales No Merch                              991780 non-null  int64         
 4   Unit Sales Any Merch                             991780 non-null  int64         
 5   Unit Sales Price Reductions Only                 991780 non-null  int64         
 6   Unit Sales Feature Only                          991780 non-null  int64         
 7   Unit Sales Display Only                          991780 non-null  int64         
 8   Unit Sales Feature and D

In [ ]:
data = data[data['Region'] != 'Total US ']

In [ ]:
data['Product Type'].unique()

array(['MEAT SUBSTITUTE', 'FISH SUBSTITUTE', 'SEAFOOD SUBSTITUTE',
       'FALAFEL', 'TEMPEH', 'POULTRY SUBSTITUTE',
       'PLANT BASED CHICKEN SUBSTITUTE', 'PLANT BASED SUBSTITUTE',
       'CHICKEN SUBSTITUTE', 'SOY SUBSTITUTE', 'TOFU', 'TOFU SUBSTITUTE',
       'SEITAN', 'PLANT BASED TURKEY SUBSTITUTE', 'BURGER',
       'TEMPEH SUBSTITUTE', 'SOY', 'VEGETABLE MEAT SUBSTITUTE'],
      dtype=object)

In [ ]:
category_mapping = {
    # General Meat Substitutes
    'MEAT SUBSTITUTE': 'PLANT BASED MEAT SUBSTITUTE',
    'PLANT BASED SUBSTITUTE': 'PLANT BASED MEAT SUBSTITUTE',
    'VEGETABLE MEAT SUBSTITUTE': 'PLANT BASED MEAT SUBSTITUTE',

    # Poultry Substitutes
    'POULTRY SUBSTITUTE': 'POULTRY SUBSTITUTE',
    'CHICKEN SUBSTITUTE': 'POULTRY SUBSTITUTE',
    'PLANT BASED CHICKEN SUBSTITUTE': 'POULTRY SUBSTITUTE',
    'PLANT BASED TURKEY SUBSTITUTE': 'POULTRY SUBSTITUTE',

    # Seafood Substitutes
    'FISH SUBSTITUTE': 'SEAFOOD SUBSTITUTE',
    'SEAFOOD SUBSTITUTE': 'SEAFOOD SUBSTITUTE',

    # Soy-Based Products
    'SOY SUBSTITUTE': 'SOY SUBSTITUTE',
    'SOY': 'SOY SUBSTITUTE',

    # Tofu and Substitutes
    'TOFU': 'TOFU',
    'TOFU SUBSTITUTE': 'TOFU',

    # Tempeh and Substitutes
    'TEMPEH': 'TEMPEH',
    'TEMPEH SUBSTITUTE': 'TEMPEH',

    # Specific Products
    'BURGER': 'BURGER',
    'SEITAN': 'SEITAN',
    'FALAFEL': 'FALAFEL'
}

# Apply the mapping to standardize the 'Category' column
data['Product Type'] = data['Product Type'].map(category_mapping)

In [ ]:
data['Product Type'].unique()

array(['PLANT BASED MEAT SUBSTITUTE', 'SEAFOOD SUBSTITUTE', 'FALAFEL',
       'TEMPEH', 'POULTRY SUBSTITUTE', 'SOY SUBSTITUTE', 'TOFU', 'SEITAN',
       'BURGER'], dtype=object)

In [ ]:
import statsmodels.formula.api as smf

# Correct the formula
formula = 'Q("Unit Sales") ~ Q("Product Type") + Region + Week + Q("Price per Unit") + Year + Week + Q("ACV Weighted Distribution") + I(Q("ACV Weighted Distribution")**2)'

# Fit the model
model = smf.ols(formula=formula, data=data).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("Unit Sales")   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                 2.673e+04
Date:                Thu, 05 Dec 2024   Prob (F-statistic):               0.00
Time:                        05:24:09   Log-Likelihood:            -7.2337e+06
No. Observations:              824367   AIC:                         1.447e+07
Df Residuals:                  824292   BIC:                         1.447e+07
Df Model:                          74                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [ ]:
model = smf.ols(
    formula="Q('Unit Sales') ~ C(Q('Product Type'), Treatment('TOFU')) * C(Region, Treatment('California ')) + C(Week, Treatment(1)) + C(Q('Total Ounces'),Treatment(1)) + C(Year, Treatment(2020)) + Q('Price per Unit') + Q('ACV Weighted Distribution') + I(Q('ACV Weighted Distribution')**2)",
    data=data,
    standardize=True
).fit()

# Print the summary
print(model.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:130: ValueWarning:

unknown kwargs ['standardize']



                            OLS Regression Results                            
Dep. Variable:        Q('Unit Sales')   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.709
Method:                 Least Squares   F-statistic:                 1.183e+04
Date:                Thu, 05 Dec 2024   Prob (F-statistic):               0.00
Time:                        05:24:36   Log-Likelihood:            -7.2289e+06
No. Observations:              824367   AIC:                         1.446e+07
Df Residuals:                  824196   BIC:                         1.446e+07
Df Model:                         170                                         
Covariance Type:            nonrobust                                         
                                                                                                                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------

In [ ]:
model_summary = model.summary().tables[1].as_html()
model_summary = pd.read_html(model_summary, header=0, index_col=0)[0]

<ipython-input-28-c264e40cc9e2>:2: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



In [ ]:
#Export the results dataframe to an Excel Sheet

model_summary.to_excel('/content/drive/MyDrive/complete_regression_results.xlsx',
                     	sheet_name='Data',
			                na_rep='N/A',
                      index = True)